### 제 2유형 연습하기. iris 종 분류 

In [31]:
# 라이브러리 불러오기
import pandas as pd 
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import f1_score, recall_score, precision_score, r2_score, mean_squared_error, accuracy_score, roc_auc_score

In [5]:
from sklearn.datasets import load_iris
iris = load_iris()
x = pd.DataFrame(iris.data, columns=['sepal_length','sepal_width','petal_length','petal_width'])
y = iris.target

# 이진 분류로 변경 (setosa = 0 , 나머지 =1)

y= np.where(y>0,1,0)

#### 데이터 탐색(EDA)
#### 데이터 전처리
#### 데이터 분리
#### 모델링 및 성능평가

In [19]:
# 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=2024)

In [ ]:
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
y_train = pd.DataFrame(y_train)

# 기존 넘파이 어레이에서 데이터 프레임으로 변경 후 컬럼명 지정
y_train.columns = ['species']

# 결측치 삽입 (아래에서 처리 연습 위해)
x_test['sepal_length'].iloc[0] = None
x_train['sepal_width'].iloc[0] = None 

# 이상치 삽입 (아래에서 처리 연습 위해)
x_train['sepal_width'].iloc[0] = 150

In [20]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(120, 4)
(120,)
(30, 4)
(30,)


In [23]:
print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 16 to 136
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  120 non-null    float64
 1   sepal_width   120 non-null    float64
 2   petal_length  120 non-null    float64
 3   petal_width   120 non-null    float64
dtypes: float64(4)
memory usage: 4.7 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 14 to 24
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  29 non-null     float64
 1   sepal_width   30 non-null     float64
 2   petal_length  30 non-null     float64
 3   petal_width   30 non-null     float64
dtypes: float64(4)
memory usage: 1.2 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   speci

In [25]:
print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
dtype: int64
sepal_length    1
sepal_width     0
petal_length    0
petal_width     0
dtype: int64
species    0
dtype: int64


In [26]:
print(x_train.describe().T)
print(x_test.describe().T)
print(y_train.describe().T)

              count      mean        std  min    25%   50%  75%    max
sepal_length  120.0  5.870833   0.843103  4.3  5.175  5.80  6.5    7.9
sepal_width   120.0  4.273333  13.420896  2.0  2.800  3.00  3.3  150.0
petal_length  120.0  3.821667   1.752270  1.0  1.600  4.35  5.1    6.7
petal_width   120.0  1.222500   0.757535  0.1  0.300  1.30  1.8    2.5
              count      mean       std  min    25%   50%    75%  max
sepal_length   29.0  5.731034  0.781986  4.8  5.100  5.60  6.200  7.7
sepal_width    30.0  3.063333  0.524229  2.2  2.725  3.00  3.375  4.4
petal_length   30.0  3.503333  1.824259  1.2  1.500  4.15  4.875  6.9
petal_width    30.0  1.106667  0.786934  0.1  0.200  1.25  1.800  2.3
         count      mean       std  min  25%  50%  75%  max
species  120.0  0.683333  0.467127  0.0  0.0  1.0  1.0  1.0


In [27]:
# 결측치 제거
# 제거 전 drop 시킬지 대체할지 결정 
x_train['sepal_length'] = x_train['sepal_length'].fillna(x['sepal_length'].median())
x_test['sepal_length'] = x_test['sepal_length'].fillna(x['sepal_length'].median())

print(x_train.isnull().sum())
print(x_test.isnull().sum())

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
dtype: int64
sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
dtype: int64


In [ ]:
# 이상치 제거 
# print(x_trian.describe().T)
# print(x_test.describe().T)

# sepal_width에서 x_train max 값 불량 확인
# Q1 = x_train['sepal_width'].quantile.(.25)
# Q3 = x_train['sepal_width'].quantile.(0.75)
# IQR = abs(Q1-Q3)
# print(Q1, Q3, IQR)

# upper = Q3+1.5*IQR
# lower = Q1-1.5*IQR

# cond1 = (x_train['sepal_width']<=upper)
# cond2 = (x_train['sepal_width]>= lower)

# print(x_train.describe().T)
# print(x_test.describe().T)

In [28]:
# 이상치 대체 (최대값으로 대체)
cond1 = (x_train['sepal_width'] <= 10)
max_sw = x_train[cond1]['sepal_width'].max()

x_train['sepal_width'] = np.where(x_train['sepal_width']>= 10, max_sw, x_train['sepal_width'])

print(x_train.describe().T)
print(x_test.describe().T)

              count      mean       std  min    25%   50%  75%  max
sepal_length  120.0  5.870833  0.843103  4.3  5.175  5.80  6.5  7.9
sepal_width   120.0  3.058333  0.419400  2.0  2.800  3.00  3.3  4.2
petal_length  120.0  3.821667  1.752270  1.0  1.600  4.35  5.1  6.7
petal_width   120.0  1.222500  0.757535  0.1  0.300  1.30  1.8  2.5
              count      mean       std  min    25%   50%    75%  max
sepal_length   30.0  5.733333  0.768489  4.8  5.100  5.65  6.200  7.7
sepal_width    30.0  3.063333  0.524229  2.2  2.725  3.00  3.375  4.4
petal_length   30.0  3.503333  1.824259  1.2  1.500  4.15  4.875  6.9
petal_width    30.0  1.106667  0.786934  0.1  0.200  1.25  1.800  2.3


In [29]:
# 모델링 적용
model = RandomForestClassifier()
model.fit(x_train, y_train)

C:\Users\chohyeri\AppData\Local\Temp\ipykernel_2156\3916786330.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)


RandomForestClassifier()

In [32]:
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test,y_pred)
f1_score = f1_score(y_test,y_pred,average='macro')
roc_auc = roc_auc_score(y_test,y_pred)

print(accuracy)
print(f1_score)
print(roc_auc)

1.0
1.0
1.0


In [33]:
y_prob = model.predict_proba(x_test)

In [34]:
result = pd.DataFrame({
    'result' : y_pred,
    'prob_0' : y_prob[:,0]
})

print(result[:5])

   result  prob_0
0       0    0.97
1       0    1.00
2       1    0.00
3       1    0.00
4       0    1.00


In [35]:
result.to_csv('test.csv',index=False)
print(pd.read_csv('test.csv').head())

   result  prob_0
0       0    0.97
1       0    1.00
2       1    0.00
3       1    0.00
4       0    1.00
